# **1️⃣ Introducción**
El ictus es una de las principales causas de discapacidad a nivel mundial, generando secuelas motoras que requieren largos periodos de rehabilitación. El seguimiento de la recuperación suele basarse en evaluaciones clínicas presenciales, escalas funcionales y observación subjetiva del desempeño del paciente, lo que limita la frecuencia y objetividad de la monitorización [1].

La electromiografía (EMG) de superficie permite registrar la actividad eléctrica de los músculos y, a partir de ella, inferir la ejecución de gestos motores. Sistemas de reconocimiento de gestos basados en EMG han demostrado ser útiles en control protésico, interfaces hombre-máquina y rehabilitación, pero la mayoría de soluciones completas son costosas, cerradas o poco adaptables a contextos de recursos limitados [2][3].

Se utilizan los algoritmos KNN y SVM para reconocer gestos con alta precisión con poca o gran cantidad de datos para la clasificación.

En este contexto, resulta relevante diseñar un sistema sencillo, reproducible y de bajo costo que procese señales EMG del antebrazo, segmente gestos de mano y permita entrenar clasificadores supervisados. Este tipo de herramienta podría emplearse como soporte para el seguimiento de la recuperación motora en pacientes con ictus, tanto en entornos clínicos como en el hogar.


# **2️⃣ Problemática**

## ***2️⃣.1️⃣Problemática General:*** 
Los pacientes con ictus necesitan sistemas asequibles que permitan realizar un seguimiento objetivo y frecuente de la recuperación motora de la mano; sin embargo, las soluciones de reconocimiento de gestos basadas en EMG disponibles en el mercado no son ampliamente accesibles por su costo, requerimientos de hardware y complejidad de uso.
## ***2️⃣.2️⃣Problemáticas Específicas:***

- Se requiere un flujo de procesamiento que segmente correctamente un registro EMG continuo en ventanas que correspondan a gestos específicos, manteniendo sincronía temporal y etiquetas claras.
- Es necesario generar un conjunto de características a partir de los canales EMG que sea adecuado para entrenar clasificadores supervisados de baja complejidad computacional (por ejemplo, K-NN o SVM), con miras a implementaciones futuras en hardware de bajo costo.

# **3️⃣ Propuesta de solución**
El sistema BioGest EMG propone generar un sistema que ayude en la rehabilitación motora mediante el uso de las señales electromiográficas (EMG), donde con las señales obtenidas se podrán reconocer 3 gestos simples de la mano del paciente post-ictus (Extensión de mano,Puño y Desviación ulnar ( girar la mano a la izquierda como haciendo un hola)), proponiendo una herramienta de bajo costo y fácil uso, integrando hardware económico, procesamiento digital de señales y algoritmos de aprendizaje automático.

Se busca automatizar la evaluación del progreso post-ictus, registrando, procesando y clasificando los movimientos de la mano en tiempo real durante las terapias, brindando retroalimentación visual y estadística sobre el progreso del paciente para una recuperación más eficiente.

Basado en estudios recientes, se demuestra la viabilidad del reconocimiento de gestos mediante sEMG multicanal, tanto con modelos de machine learning tradicionales (Random Forest, SVM, kNN), como con enfoques de deep learning basados en CNN y espectrogramas [4].

El flujo de trabajo que sigue la propuesta es:
- *Segmentación temporal y generación de archivos por gesto:* Se parte de un archivo CSV con todos los canales EMG de un experimento, en donde cada experimento se organiza en ciclos que contienen 10 gestos; a su vez, cada gesto ocupa una ventana de 10 s: 4 s de reposo inicial y 6 s de gesto activo. El script automatiza el corte de un ciclo en 10 ventanas de 10 s, una por gesto, combinando todos los canales EMG en una única señal y generando archivos separados. 
- *Extracción de características y clasificación:* A partir de cada archivo de gesto se pueden calcular características clásicas en el dominio temporal (RMS, valor absoluto medio, varianza, etc.) y/o en frecuencia. Se utiliza un clasificador supervisado, como K-NN o SVM, para distinguir entre 3 gestos de mano a partir de estas características. El objetivo es obtener un porcentaje de clasificación correcto suficientemente alto como para que el sistema sea útil en un escenario de seguimiento motor.

![Imagen de proyecto 1](ISB_Proyecto_1.jpeg)
Fig 01 - Página donde se realizó el trabajo: Edge Impulse

![Imagen de proyecto 2](ISB_Proyecto_2.jpeg)
Fig 02 - Parámetros de procesamiento de data: espectrograma y su resultado

![Imagen de proyecto 3](ISB_Proyecto_3.jpeg)

Fig 03 - Parámetros de la red neuronal

# **4️⃣ Código**

In [ ]:
import pandas as pd
import numpy as np
import os

# ==========================
# RUTAS
# ==========================
INPUT_FILE = "/Users/lucianatarazona/Desktop/proyecto/csv/1_filtered.csv"
OUT_DIR    = "/Users/lucianatarazona/Desktop/proyecto/participante_1_2"

# ==========================
# PARÁMETROS
# ==========================
FS = 2000          # Frecuencia de muestreo (Hz)  <-- cámbiala si es otra
NUM_GESTURES = 10  # 10 gestos en el ciclo
REST_SEC     = 4.0 # 4 s de reposo inicial
GESTURE_SEC  = 6.0 # 6 s de gesto
GESTO_TOTAL_SEC = REST_SEC + GESTURE_SEC  # 10 s útiles por gesto

# ==========================
# LEER CSV COMPLETO
# ==========================
# header=None para no perder la primera fila de datos
df = pd.read_csv(INPUT_FILE, header=None)

# Nombrar columnas como ch1, ch2, ...
num_channels = df.shape[1]
channel_names = [f"ch{i+1}" for i in range(num_channels)]
df.columns = channel_names

total_samples = len(df)
total_sec = total_samples / FS

# Sabemos que hay 5 ciclos en total
NUM_CYCLES_TOTAL = 5
cycle_sec = total_sec / NUM_CYCLES_TOTAL   # duración de 1 ciclo (= 10 gestos de 10 s + 4 s de reposo final)

print(f"Duración total: {total_sec:.2f} s")
print(f"Duración estimada de 1 ciclo: {cycle_sec:.2f} s")

# SOLO usamos el primer ciclo
cycle_start_sec = 0.0

# Crear carpeta de salida
os.makedirs(OUT_DIR, exist_ok=True)

# ==========================
# CORTAR SOLO 1 CICLO (10 GESTOS)
# ==========================
for g in range(NUM_GESTURES):
    # Inicio del bloque útil (reposo inicial + gesto) de este gesto
    # Los gestos se suceden cada 10 s dentro del ciclo:
    # [4s reposo + 6s gesto] * 10  +  4s reposo final (que ignoramos)
    gesture_start_sec = cycle_start_sec + g * GESTO_TOTAL_SEC
    gesture_end_sec   = gesture_start_sec + GESTO_TOTAL_SEC  # 10 s totales

    # Pasar a índices
    start_idx = int(round(gesture_start_sec * FS))
    end_idx   = int(round(gesture_end_sec   * FS))

    # Seguridad
    start_idx = max(start_idx, 0)
    end_idx   = min(end_idx, total_samples)

    segment = df.iloc[start_idx:end_idx].reset_index(drop=True)

    # ==========================
    # COMBINAR CANALES → EMG (RMS)
    # ==========================
    # Si solo quieres los 4 primeros canales, descomenta esta línea:
    # segment_for_emg = segment.iloc[:, :4]
    segment_for_emg = segment  # usa todas las columnas como EMG

    # RMS por muestra
    emg_signal = np.sqrt((segment_for_emg ** 2).sum(axis=1))

    # ==========================
    # TIMESTAMP EN ms (0–10 s)
    # ==========================
    n = len(emg_signal)
    dt_ms = 1000.0 / FS      # intervalo en milisegundos
    timestamps = np.arange(n) * dt_ms

    out_df = pd.DataFrame({
        "timestamp": timestamps,
        "emg": emg_signal
    })

    # Guardar archivo de este gesto (solo 1 ciclo, 10 s)
    filename = os.path.join(OUT_DIR, f"gesture_{g+1:02d}.csv")
    out_df.to_csv(filename, index=False)
    print(f"Gesto {g+1} guardado en: {filename}")

## 4️⃣.1️⃣ Importación de librerías y definición de rutas

In [ ]:
import pandas as pd
import numpy as np
import os

INPUT_FILE = "....../1_filtered.csv"
OUT_DIR    = "....../participante_1_2"

- Se importan pandas y numpy para manejar datos y operaciones numéricas, y os para gestionar rutas y carpetas.

- INPUT_FILE apunta al CSV con la señal EMG ya filtrada; OUT_DIR es la carpeta donde se guardarán los gestos segmentados.

## 4️⃣.2️⃣ Definición de parámetros del experimento

In [ ]:
FS = 2000          # Frecuencia de muestreo (Hz)
NUM_GESTURES = 10  # 10 gestos en el ciclo
REST_SEC     = 4.0 # 4 s de reposo inicial
GESTURE_SEC  = 6.0 # 6 s de gesto
GESTO_TOTAL_SEC = REST_SEC + GESTURE_SEC  # 10 s por gesto

- FS fija la frecuencia de muestreo en 2000 Hz.

- NUM_GESTURES indica que en un ciclo hay 10 gestos.

- Cada gesto está compuesto por 4 s de reposo y 6 s de ejecución, sumando 10 s (GESTO_TOTAL_SEC)

## 4️⃣.3️⃣ Lectura del archivo CSV y nombrado de canales

In [ ]:
df = pd.read_csv(INPUT_FILE, header=None)
num_channels = df.shape[1]
channel_names = [f"ch{i+1}" for i in range(num_channels)]
df.columns = channel_names

- pd.read_csv carga todo el registro EMG, que inicialmente no tiene encabezados.

- Se cuentan las columnas (num_channels) y se les asignan nombres genéricos (ch1, ch2, ..., chN), representando cada uno de los canales EMG registrados.

## 4️⃣.4️⃣ Cálculo de duración total y del ciclo

In [ ]:
total_samples = len(df)
total_sec = total_samples / FS

NUM_CYCLES_TOTAL = 5
cycle_sec = total_sec / NUM_CYCLES_TOTAL

- total_samples es el número de muestras en todo el registro, y total_sec su duración en segundos.

- Se asume que el experimento contiene 5 ciclos completos (NUM_CYCLES_TOTAL), por lo que la duración de cada ciclo se estima como cycle_sec = total_sec / 5.

## 4️⃣.5️⃣ Selección del ciclo a procesar y creación de carpeta de salida

In [ ]:
cycle_start_sec = 0.0
os.makedirs(OUT_DIR, exist_ok=True)

- cycle_start_sec indica desde qué segundo del registro se toma el ciclo (en este caso, se usa el primer ciclo, empezando en 0 s).

- os.makedirs crea la carpeta de salida si no existe (exist_ok=True evita errores si la carpeta ya fue creada).

## 4️⃣.6️⃣ Bucle principal de segmentación de gestos

In [ ]:
for g in range(NUM_GESTURES):
    gesture_start_sec = cycle_start_sec + g * GESTO_TOTAL_SEC
    gesture_end_sec   = gesture_start_sec + GESTO_TOTAL_SEC
    ...

- Se recorre cada gesto g de 0 a 9 (10 gestos).

- Para cada gesto se calcula el tiempo de inicio (gesture_start_sec) y fin (gesture_end_sec) dentro del ciclo, separados por bloques de 10 s.

## 4️⃣.7️⃣ Conversión de tiempos a índices de muestra y corte de la señal

- Se multiplican los tiempos por la frecuencia de muestreo para obtener índices en la señal (start_idx, end_idx).

- Se aplican límites de seguridad para no salir del rango del dataframe.

- segment contiene la porción de la señal correspondiente a los 10 s (reposo + gesto) para ese gesto.

## 4️⃣.8️⃣ Combinación de canales EMG en una sola señal (tipo RMS/vector magnitud)

In [ ]:
segment_for_emg = segment  # se usan todos los canales
emg_signal = np.sqrt((segment_for_emg ** 2).sum(axis=1))

- segment_for_emg selecciona los canales a combinar. En el comentario se indica que, si se desea, se pueden usar solo algunos canales (por ejemplo, los 4 primeros).

- Se calcula, muestra a muestra, la raíz cuadrática de la suma de cuadrados de los canales. Esto genera una señal emg_signal que representa la magnitud global de la actividad EMG en el antebrazo, reduciendo la dimensionalidad y facilitando el procesamiento posterior.

## 4️⃣.9️⃣ Generación de marcas de tiempo y construcción del DataFrame de salida

In [ ]:
n = len(emg_signal)
dt_ms = 1000.0 / FS
timestamps = np.arange(n) * dt_ms

out_df = pd.DataFrame({
    "timestamp": timestamps,
    "emg": emg_signal
})

- dt_ms es el intervalo de tiempo en milisegundos entre muestras.

- timestamps es un vector que va de 0 a (n-1)·dt_ms, que marca el tiempo de cada muestra dentro de la ventana de 10 s.

- Se crea un nuevo DataFrame out_df con dos columnas: timestamp (en ms) y emg (magnitud de la señal).

## 4️⃣.🔟 Guardado de cada gesto en un archivo CSV independiente

In [ ]:
filename = os.path.join(OUT_DIR, f"gesture_{g+1:02d}.csv")
out_df.to_csv(filename, index=False)

- Se construye un nombre de archivo para cada gesto (gesture_01.csv, gesture_02.csv, …, gesture_10.csv).

- Cada archivo contiene la señal EMG combinada y su marca de tiempo, lista para ser usada en las etapas de extracción de características y entrenamiento del clasificador.

# **5️⃣ Resultados**

![Imagen de proyecto 4](ISB_Proyecto_4.jpeg)

Fig 04 - Matriz de confusión con accuracy y pérdida

![Imagen de proyecto 5](ISB_Proyecto_5.jpeg)

Fig 05 - Clasificación de datos

# **6️⃣ Conclusiones**

El diseño e implementación del sistema BioGest EMG permitió validar, en primera instancia, la eficacia del flujo de procesamiento digital desarrollado. El algoritmo de segmentación automatizada logró fragmentar exitosamente los registros continuos en ventanas temporales de 10 segundos, garantizando la sincronía necesaria entre los periodos de reposo y la actividad muscular para la correcta generación del conjunto de datos. De igual manera, la estrategia de preprocesamiento basada en la señal de magnitud global (RMS) demostró ser altamente eficiente; al sintetizar la información de múltiples canales en un único vector, se redujo significativamente la carga computacional sin comprometer la integridad de la información muscular, lo cual resulta fundamental para la viabilidad de futuras implementaciones en hardware de recursos limitados.

Por otro lado, la evaluación experimental confirma la capacidad del sistema para mitigar la subjetividad inherente a las evaluaciones clínicas tradicionales. Al proporcionar un método objetivo para cuantificar la ejecución de gestos funcionales como la extensión de mano y la desviación ulnar, la herramienta facilita un seguimiento más riguroso de la recuperación motora en pacientes con secuelas de ictus. Esto se ve respaldado por el desempeño de los algoritmos de clasificación, cuyos resultados —visualizados en las matrices de confusión— evidencian una alta discriminabilidad entre los estados de reposo y los gestos activos, validando así el uso de modelos supervisados para el monitoreo en entornos de rehabilitación.

# **7️⃣ Bibliografía**

[1]M. A. Ozdemir, D. H. Kisa, O. Guren, A. Onan, and A. Akan, “EMG based Hand Gesture Recognition using Deep Learning,” in 2020 Medical Technologies National Congress (TIPTEKNO), Antalya, Turkey, 2020, pp. 455–459. doi: 10.1109/TIPTEKNO50054.2020.9299264

[2] Ministerio de Salud (MINSA), Accidente cerebrovascular es la primera causa de discapacidad permanente en edad adulta. [En línea]. Disponible en: https://www.gob.pe/institucion/minsa/noticias/61387-accidente-cerebrovascular-es-la-primera-causa-de-discapacidad-permanente-en-edad-adulta.

[3]Ministerio de Salud (MINSA), Minsa: 1 de cada 4 personas sufrirá un ataque cerebrovascular durante su vida. [En línea]. Disponible en: https://www.gob.pe/institucion/minsa/noticias/1274851-minsa-1-de-cada-4-personas-sufrira-un-ataque-cerebrovascular-durante-su-vida

[4] Asociación Daño Cerebral Adquirido, ¿Y si la recuperación del miembro superior tras un ictus dejara de ser una excepción?. [En línea]. Disponible en: https://xn--daocerebral-2db.es/y-si-la-recuperacion-del-miembro-superior-tras-un-ictus-dejara-de-ser-una-excepcion/#:~:text=Una%20secuela%20frecuente,terapeuta%20ocupacional%20en%20la%20introducci%C3%B3n.

[5] M. A. Ozdemir, D. H. Kisa, O. Guren, and A. Akan, “Dataset for multi-channel surface electromyography (sEMG) signals of hand gestures,” Mendeley Data, Version 2, Dec. 22, 2021. data: https://data.mendeley.com/datasets/ckwc76xr2z/2